# Lab 7 - Elementy Programowania Funkcyjnego

## Podstawy

Program składa się z **czystych funkcji**, czyli funkcji bez **efektów ubocznych**.

- Konstrukty odnoszą się do pojedynczych obiektów lub pojedynczej kolekcji obiektów
- Wykorzystywanie funkcji jako parametrów funkcji
- Rzadkie wykorzysywanie pętli
- Wykorzystanie `filter`, `map`, `reduce`
- Lambdy

### Podstawowe cechy

- podstawowa składnia funkcji `([parameters]) -> [result-type]`

- funkcja jest typem pierwszoklasowym - jest konstruktem służącym do przechowywania danych, na którym możemy wykonywać takie same operacje, jak na innych, wbudowanych typach

In [ ]:
val f: (Int, String) -> String = { i:Int, s:String -> "${i}: ${s}" } // function literal
val f2 = { i:Int, s:String -> "${i}: ${s}" } // anonymous lambda

fun ff(fun1: (Int,String) -> String):String {
 return fun1(7, "Hello")
}
ff(f)

In [ ]:
ff( { i:Int,s:String -> "${i}- ${s}" } )

- wywołanie ` function({ [lambda-function] })` może zostać skócone do `function { [lambda-function] }`

In [ ]:
ff { i:Int,s:String -> "${i}- ${s}" }

In [ ]:
object X {
 fun add(a:Int, b:Int): Int = a + b
}

val f : (Int,Int) -> Int = X::add

In [ ]:
X.add(2, 2)

In [ ]:
f(2, 3)

- odniesienie do funkcji z klas i obiektów przez `::`

In [ ]:
class X2 {
 fun add(a:Int, b:Int): Int = a + b
}

val f : X2.(Int,Int) -> Int = X2::add
f(2, 2)

- odniesienie do metod instancji przez `::`

In [ ]:
class X3 {
 fun add(a:Int, b:Int): Int = a + b
}

val x3 = X3()
val f : (Int,Int) -> Int = x3::add
f(2, 2)

In [ ]:
val f : (Int,Int) -> Int = X3::add

In [ ]:
X3.add(2, 2)

In [ ]:
x3.add(2, 2)

## Pętle

Pętle można zrealizować przez zastosowanie rekursji z funkcją pomocniczą. Typowo nazywa się `go` lub `loop`. Definicja `factorial` zawiera tylko wywołanie `go` z początkowymi warunkami pętli.

In [ ]:
fun factorial(i: Int): Int {
    fun go(n: Int, acc: Int): Int =
        if (n <= 0) acc
        else go(n - 1, n * acc)
    return go(i, 1)
}

factorial(9)

In [ ]:
fun factorial2(i: Int): Int {
    tailrec fun go(n: Int, acc: Int): Int =
        if (n <= 0) acc
        else go(n - 1, n * acc)
    return go(i, 1)
}

factorial2(9)

Przyjmuje się, że wywołanie znajduje się na `tail`, jeśli wywołujący nie robi nic poza zwrotem wartości rekurencyjnego wywołania.<br>
Przykładowo:<br>
- `go(n-1, n*acc)` jest w pozycji `tail` ponieważ metoda zwraca wartość wywołania i nic z nim więcej nie robi
- `1 + go(n-1, n*acc)` nie jest w pozycji `tail` ponieważ po zwróceniu `go` dalej wykonywana jest operacja dodawania<br>
Jeżeli wszystkie wywołania rekursywne są w pozycji `tail` zastosowanie modyfikatora `tailrec` powoduje zastąpienie rekursji wywołaniami iteratywnymi przez kompilator. Dzięki temu unikamy `StackOverflowError`.

## Pętle vs Rekurencja

Wykorzystanie funkcji aby pozbyć się mutowalności iteracyjnej

Pętle wykorzystują efekty uboczne

In [40]:
val a = listOf(1, 2, 3, 4)

fun suma(l: List<Int>): Int{ // czysta funkcja
    var sum = 0
    for (i in l){
        sum += i // efekt uboczny - modyfikacja stanu
    }
    return sum
}
println(suma(a))

10


## Inline fun

In [1]:
class A {
    fun function1(i:Int, f:(Int) -> String): String {
        return f(i)
    }
    
    fun function2() {
        val a = 7
        val s = function1(8) {
            i -> "8 + a = " + (i+a) 
        }
    }
}

W wywołaniu `function1` przekazujemy funkcje w postaci lambdy, jest to obiekt który musi zostać utworzony w czasie wykonania. Dodatkowo właściwość `a` musi zostać przekazana do tego obiektu.

In [2]:
class A {
    inline fun function1(i:Int, f:(Int) -> String): String
    {
        return f(i)
    }
    fun function2() {
        val a = 7
        val s = function1(8) {
            i -> "8 + a = " + (i+a) }
    }
}

Gdy funkcja `inline` zostaje wykorzystana, cały kod funkcji zostaje skopiowany w miejsce wywołania

Prawie wszystkie funkcje wyższego rzędu w `stdlib` są `inline`

In [3]:
inline fun repeat(times: Int, action: (Int) -> Unit) {
    for (index in 0 until times) {
        action(index)
    }
}

In [4]:
repeat(10) {
    print(it)
}

0123456789

Powyższe wywołanie zostanie zastąpione przez

In [5]:
for (index in 0 until 10) {
    print(index)
}

0123456789

Wykonanie funkcji przyjmujące funkcję jako parametr są szybsze gdy są `inline`.

## Funkcje infix

- muszą być składowymi klasy lub funkcjami rozszerzającymi
- muszą posiadać jeden parametr
- parametr nie może posiadać wartości domyślnej
- parametr nie może mieć akceptować `varargs`

In [102]:
class MyString(val str: String) {
    infix fun ad(s: String): String { return str + s }
}

In [103]:
val myStr = MyString("Rafał")

In [105]:
myStr ad " Lewandków"

Rafał Lewandków

## Funkcje wyższego rzędu

In [ ]:
object Example {
    private fun abs(n: Int): Int =
        if (n < 0) -n
        else n
    
    private fun factorial(i: Int): Int {
        fun go(n: Int, acc: Int): Int =
            if (n <= 0) acc
            else go(n - 1, n * acc)
        return go(i, 1)
}
    
    fun formatAbs(x: Int): String {
        val msg = "The absolute value of %d is %d"
        return msg.format(x, abs(x))
    }

    fun formatFactorial(x: Int): String {
        val msg = "The factorial of %d is %d"
        return msg.format(x, factorial(x))
    }
}

fun main() {
    println(Example.formatAbs(-42))
    println(Example.formatFactorial(7))
}

main()

wprowadzamy funkcję wyższego rzędu

In [ ]:
fun formatResult(name: String, n: Int, f: (Int) -> Int): String {
    val msg = "The %s of %d is %d."
    return msg.format(name, n, f(n))
}

In [ ]:
fun main() {
    println(formatResult("factorial", 7, ::factorial))
    println(formatResult("absolute value", -42, ::abs))
}
main()

In [ ]:
formatResult(
    "absolute", 
    -42,
    fun(n: Int): Int { return if (n < 0) -n else n } // funkcja anonimowa
)

In [ ]:
formatResult(
    "absolute", 
    -42, 
    { n -> if (n < 0) -n else n } // lambda anonimowa
)

In [ ]:
formatResult("absolute", -42) 
    { if (it < 0) -it else it }

## Funkcje Monomorficzne i Polimorficzne

Funkcja `factorial` jest funkcją monomorficzną, przyjmuje tylko jeden typ argumentu.

Poniżej funkcja **monomorficzna** `findFirst` zwracająca indeks pierwszego wystąpienia elementu w tabeli `String`.

In [ ]:
fun findFirstMonomorphic(ss: Array<String>, key: String): Int {
    tailrec fun loop(n: Int): Int =
        when {
            n >= ss.size -> -1
            ss[n] == key -> n
            else -> loop(n + 1)
        }
    return loop(0)
}

val a = arrayOf("Rafał", "Robert", "Robert")
val s = "Robert"
findFirstMonomorphic(a, s)

In [ ]:
val a = arrayOf(1, 2, 3, 4)
val s = 2
findFirstMonomorphic(a, s)

Taką funkcję możemy zmienić na funkcję **polimorficzną** pracującą na argumencie typowalnym.

In [ ]:
fun <A> findFirstPolimorphic(xs: Array<A>, p: (A) -> Boolean): Int 
{
    tailrec fun loop(n: Int): Int =
        when {
            n >= xs.size -> -1
            p(xs[n]) -> n
            else -> loop(n + 1)
    }
        
    return loop(0)
}

val a = arrayOf("Rafał", "Robert", "Robert", "Paweł")
findFirstPolimorphic(a) { it == "Paweł"}

In [ ]:
findFirstPolimorphic(arrayOf(1, 2, 3, 4)){it == 4}

Funkcja **polimorficzna** jest funkcją **generyczną**.

## Funkcja przyjmująca jako parametr funkcję - Java

In [2]:
interface Callable {
    public void call(int param);
}
    
class Test implements Callable {
    public void call(int param) {
        System.out.println("Called with " + param );
    }
}
    
public class HelloWorld{
    public static void invoke(Callable callable, int param){
        callable.call(param);
    }
    
     public static void main(){
        // Callable cmd = new Test();
        // invoke(cmd, 100);
         
         // klasa anonimowa
        // invoke(new Callable(){
        //     public void call(int param){
        //         System.out.println("Called with " + param );
        //     }
        // }, 100);
         
        //invoke((param -> System.out.println("Called with " + param )), 100);

    }
}
HelloWorld.main()

Called with 100


In [6]:
import java.util.function.Function;

public class HigherOrderFunc {

    public static void main() {

        //Function <Integer, Integer> inc = e -> e + 1;
        doSum(5, e -> e + 1);

    }

    public static void doSum(int value, Function <Integer, Integer> func) {
        System.out.println(func.apply(value));
    }
}

HigherOrderFunc.main()

6
